## Importing Data and Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import ast

In [ ]:
closeddf = pd.read_csv('closed.csv')
opendf = pd.read_csv('openrest.csv')

## Dropping less relevant rows and columns and combining the data

In [ ]:
#dropping columns from the open restaurants data with very low numbers of reviews in order to reduce the risk or using businesses that have opened since the pandemic

def drop_low_counts(dataframe):
    h = []
    for i in np.where(dataframe.review_count < 16):
        h.append(i)
    return opendf.drop(index = h[0], inplace = True)

drop_low_counts(opendf)

In [ ]:
df = pd.concat([closeddf, opendf], axis = 0)

In [ ]:
#dropping unused columns

df.drop(columns = ['Unnamed: 0','distance', 'id', 'image_url','phone'], inplace = True)

#randomizing and re-indexing the rows

df = df.sample(frac=1).reset_index(drop=True)

## Cleaning price column 
#### converting dollar sign to categorical numeric variable

In [ ]:
df['price'].value_counts()

In [ ]:
df.isna().sum()

In [ ]:
cond = [df['price'] == '$',
        df['price'] == '$$',
        df['price'] == '$$$',
        df['price'] == '$$$$'
       ]
choice = ['one','two','three','four']
df['price_cat'] = np.select(cond, choice,default='missing')
df.head()

In [ ]:
df['price_cat'].value_counts()

In [ ]:
price_dummies = pd.get_dummies(data = df['price'])

## Breaking out transactions column

In [ ]:
df['transactions'] = df['transactions'].map(lambda x: ast.literal_eval(x))

In [ ]:
df['transactions'][1]

In [ ]:
df['delivery'] = df['transactions'].apply(lambda x: 1 if 'delivery' in x else 0)
df['restaurant_reservation'] = df['transactions'].apply(lambda x: 1 if 'restaurant_reservation' in x else 0)
df['pickup'] = df['transactions'].apply(lambda x: 1 if 'pickup' in x else 0)

In [ ]:
df.head(5)

In [ ]:
print(df['delivery'].value_counts())
print(df['restaurant_reservation'].value_counts())
print(df['pickup'].value_counts())

## Pull out zip from the location column

In [ ]:
df['location'] = df['location'].map(lambda x: ast.literal_eval(x))

In [ ]:
df['location']

In [ ]:
df['location'][0]

In [ ]:
df['location'][0]['zip_code']

In [ ]:
df['zip_code'] = df['location'].apply(lambda x: x['zip_code'])

## Pulling out lat and long

In [ ]:
df['coordinates'] = df['coordinates'].map(lambda x: ast.literal_eval(x))

In [ ]:
df['coordinates']

In [ ]:
df['latitude'] = df['coordinates'].map(lambda x: x['latitude'])
df['longitude'] = df['coordinates'].map(lambda x: x['longitude'])

## Reformatting and binning restaurant categories

In [ ]:
def reformat_categories(column):
    y = []
    for i in column:
        try:
            y.append(i.strip('][').split(', ')[3][10:-2])
        except IndexError:
            y.append(i.strip('][').split(', ')[1][10:-2])
    return y

df['categories'] = reformat_categories(df.categories)

In [ ]:
#replacing one oddly formatted entry

df['categories'] = df.categories.replace("'cafe", 'Cafes')

#dropping a few non-restaurants that made it into our dataset

df = df[df['categories'] != 'Souvenir Shops']
df = df[df['categories'] != 'Candy Stores']
df = df[df['categories'] != 'Print Media']

In [ ]:
#creating a mini DataFrame of categories the permanently closed dummy

closedcat = df.groupby(['categories'])['permanently closed'].value_counts().unstack().fillna(0)

In [ ]:
#adding a percentage column to closedcat

closedcat['percentclosed'] = (round(closedcat[1]/(closedcat[0]+closedcat[1]), 2)*100).astype(int)

In [ ]:
#this is a mess and we need to bin some categories

fig, ax = plt.subplots(figsize = (15, 9))
plt.barh(y = closedcat.index, width = closedcat.percentclosed);

In [ ]:
df.categories.value_counts()

In [ ]:
df['categories'] = df.categories.replace(['Gay Bars', 'Dive Bars', 'Sports Bars', 'Wine Bars', 'Champagne Bars', 'Cocktail Bars', 'Beer Bar', 'Whiskey Bars', 'Karaoke', 'Lounges', 'Dance Clubs'], 'Bars')

df['categories'] = df.categories.replace(['Coffee & Tea', 'Tea Rooms', 'Juice Bars & Smoothies', 'Bubble Tea', 'Furniture Stores', 'Coffee Roasteries', 'Pet Stores'], 'Cafes')

df['categories'] = df.categories.replace(['Burgers', 'Sandwiches', 'Comfort Food', 'Diners', 'Southern', 'Salad', 'Pubs', 'Chicken Wings', 'Delis', 'Hot Dogs', 'Fast Food', 'Cheesesteaks', 'Barbeque', 'Soul Food', 'Soup', 'Wraps', 'Buffets', 'Smokehouse', 'American (Traditional)', 'Chicken Shop'], 'American (Casual)')

df['categories'] = df.categories.replace(['Gastropubs'], 'American (New)')

df['categories'] = df.categories.replace(['Szechuan', 'Cantonese', 'Hot Pot', 'Shanghainese', 'Dim Sum', 'Hong Kong Style Cafe'], 'Chinese')

df['categories'] = df.categories.replace(['Bakeries', 'Desserts', 'Ice Cream & Frozen Yogurt', 'Cupcakes'], 'Bakeries and Desserts')

df['categories'] = df.categories.replace(['Trinidadian', 'Cuban'], 'Caribbean')

df['categories'] = df.categories.replace(['Sushi Bars', 'Ramen', 'Izakaya', 'Japanese Curry'], 'Japanese')

df['categories'] = df.categories.replace(['Italian', 'French', 'Tapas/Small Plates', 'Pasta Shops', 'Irish', 'German', 'Sicilian', 'Sardinian', 'Modern European', 'Brasseries', 'Scottish', 'British', 'Fish & Chips', 'Basque', 'Portuguese', 'Tuscan', 'Austrian', 'Spanish', 'Tapas Bars'], 'European')

df['categories'] = df.categories.replace(['Mediterranean', 'Greek', 'Middle Eastern', 'Falafel', 'Turkish', 'Kebab', 'Halal', 'Persian/Iranian', 'Lebanese'], 'Middle Eastern/Mediterranean')

df['categories'] = df.categories.replace(['Tacos', 'New Mexican Cuisine', 'Tex-Mex'], 'Mexican')

df['categories'] = df.categories.replace(['Empanadas', 'Puerto Rican', 'Salvadoran', 'Dominican', 'Colombian', 'Peruvian', 'Brazilian', 'Venezuelan'], 'Latin American')

df['categories'] = df.categories.replace(['Laotian', 'Taiwanese', 'Malaysian', 'Thai', 'Singaporean', 'Burmese', 'Vietnamese', 'Pan Asian', 'Indonesian', 'Filipino', 'Asian Fusion'], 'South Asian') 

df['categories'] = df.categories.replace(['Ethiopian', 'Senegalese', 'Somali', 'Moroccan'], 'African') 

df['categories'] = df.categories.replace(['Creperies', 'Waffles', 'Bagels', 'Pancakes', 'Donuts'], 'Breakfast & Brunch')

df['categories'] = df.categories.replace(['Vegetarian', 'Vegan', 'Kosher', 'Gluten-Free'], 'Specialty Food')

df['categories'] = df.categories.replace(['Russian', 'Polish', 'Georgian', 'Ukrainian', 'Himalayan/Nepalese'], 'Eastern Europe/Central Asia')

df['categories'] = df.categories.replace(['Public Markets', 'Performing Arts', 'Food Trucks', 'Food Court', 'Festivals', 'Venues & Event Spaces', 'Music Venues', 'Caterers', 'Food Stands', 'Jazz & Blues', 'Food Delivery Services'], 'Mobile, Events and Venues')

df['categories'] = df.categories.replace(['Indian', 'Hawaiian', 'Australian', 'Cajun/Creole', 'International Grocery', 'Grocery', 'Fondue', 'Poke'], 'Misc')

In [ ]:
#remaking the closedcat dataframe now that everything is binned

closedcat = df.groupby(['categories'])['permanently closed'].value_counts().unstack().fillna(0)

closedcat['percentclosed'] = (round(closedcat[1]/(closedcat[0]+closedcat[1]), 2)*100).astype(int)

In [ ]:
fig, ax = plt.subplots(figsize = (15, 9))
plt.barh(y = closedcat.index, width = closedcat.percentclosed);

## Importing the DOH restaurant data to find the number of restaurants in each zip code

In [ ]:
restsdf = pd.read_csv('Restaurants__rolled_up_.csv')

In [ ]:
restsdf.head()

In [ ]:
zipdf = pd.DataFrame(restsdf.ZIPCODE.value_counts()).reset_index()
zipdf.columns = ['zip_code', 'total_restaurants_in_zip']

In [ ]:
zipdf['zip_code'] = zipdf.zip_code.astype(int)
zipdf['zip_code'] = zipdf.zip_code.astype(str)

In [ ]:
df = df.merge(zipdf, how = 'left', on = 'zip_code')

## Importing population density by zipcode

In [ ]:
popdens = pd.read_excel('pop density by zip ny.xlsx')

In [ ]:
popdens['zip_code'] = popdens['Zip / Population'].apply(lambda x: x[:5])

In [ ]:
popdens['Population Density '] = popdens['Population Density '].apply(lambda x: x.replace('/sq mi', ''))
popdens['Population Density '] = popdens['Population Density '].apply(lambda x: x.replace(',', ''))

In [ ]:
popdens = popdens[['zip_code', 'Population Density ']]

In [ ]:
df = df.merge(popdens, how = 'left', on = 'zip_code')

In [ ]:
df.to_csv('mergeddate.csv')